In [1]:
# import libraries
import subprocess
import json
import os
from constants import *
from dotenv import load_dotenv
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

load_dotenv()

True

In [2]:
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

witness injury spoon lumber refuse strike shrimp calm prefer horse stumble infant zebra merry grow


In [3]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [4]:
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    command = './derive -g --mnemonic="{mnemonic}" --coin="btc-test" --cols=path,address,privkey,pubkey --format=json'
    if coin == "eth":
        command = './derive -g --mnemonic="{mnemonic}" --coin="eth" --cols=path,address,privkey,pubkey --format=json'
    if coin == "btc":
        command = './derive -g --mnemonic="{mnemonic}" --coin="btc" --cols=path,address,privkey,pubkey --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [5]:

# command = './derive -g --mnemonic="{mnemonic}"  --coin="btc-test" --cols=path,address,privkey,pubkey --format=json'
# p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
# (output, err) = p.communicate()
# p_status = p.wait()
# keys = json.loads(output)
# print(keys)

In [6]:
# Test the function derive_wallets
#derive_wallets(mnemonic, 'btc', 3)
#derive_wallets(mnemonic, 'eth',3)

In [7]:
#Setting dictionary of coins to be used in the wallet

coins = {"eth", "btc-test", "btc"}
numderive = 3

In [8]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('MNEMONIC'), coin, numderive)

In [9]:
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']

print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0x9d560f1207789c19a9fbb1c4cf6c8f5828d5b2219034cc7362a3705ab20f2305"
"cV3EfoUo5TJfbS356NhsUB9yTwCcb5YddgCva45uUPHeR1jUuZWF"


In [10]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "15Ji2eBZxbjw7kpeU7A2KXRs2k6wHbaESS",
            "path": "m/44'/0'/0'/0/0",
            "privkey": "KzfiLVcitrrvW7jUcwxKbEHSjsXDKq47X5w3r7uh8DJTThmjZD3R",
            "pubkey": "02fecd1c9636ba4663df6d72b2c3959bee27530807af098173938c608e3c34cd18"
        },
        {
            "address": "1BVCTgqVXuVUugbJVYi2SVAn1WivewzPT1",
            "path": "m/44'/0'/0'/0/1",
            "privkey": "KxJrcUcg9MyeGmJF8M3QzQ1d5BwzJxj5ErqwbJ4EkapQBzx9RZtx",
            "pubkey": "029ff90cfd29575b78928a39a22ba490236c26044d80584748a5e88ebea81a7c9f"
        },
        {
            "address": "1HAWgDGTxgd44gb97Hw4KGxuCmnzTPRU4A",
            "path": "m/44'/0'/0'/0/2",
            "privkey": "L4VhtgSRBAQ3zWTd72H5F8MqW1R1E8bTJq3aAxVqzQBrAS5nWnx4",
            "pubkey": "02fac020fed67ceaaa4f8e138a48a2b77334ec7db239f3ecd6584cff1d529938a5"
        },
        {
            "address": "13HZsdcLSeF8bsLfp8ZU5MeeKyHd7qrLE1",
            "path": "m/44'/0'/0'/0/3",
  

In [21]:
def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [22]:
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

btc-test
cV3EfoUo5TJfbS356NhsUB9yTwCcb5YddgCva45uUPHeR1jUuZWF


In [23]:
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)

eth
0x9d560f1207789c19a9fbb1c4cf6c8f5828d5b2219034cc7362a3705ab20f2305


In [24]:
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [25]:
# create a function to send txn
def send_tx(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [29]:
# create BTC transaction
create_tx(BTCTEST,btc_acc,"mkR3RZ8U2RbaPEakrtwQnoK8jSmhFmEYku", 0.1)

In [30]:
#send BTC transaction
send_trx(BTCTEST,btc_acc,'miEz4gE7RfSDXgzrSwij62a2GFyrPmJsFV',0.1)

In [40]:
#connecting to HTTP with address pk
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x9d560f1207789c19a9fbb1c4cf6c8f5828d5b2219034cc7362a3705ab20f2305"))

In [41]:
# double check if  I am connected to blockchain. 
w3.isConnected()

False

In [42]:
#checking the Block Number
w3.eth.blockNumber

In [ ]:
w3.eth.getBalance("0x503B3648E16a2103Edd7f9A768601c865714B492")

In [ ]:
create_trx(ETH,eth_acc,"0x503B3648E16a2103Edd7f9A768601c865714B492", 1000)

In [ ]:
send_trx(ETH, eth_acc,"0xbFDf092Ea03FC2ff3f9D0a8aFf0F567638209C0F", 1000)

In [ ]:
w3.eth.getBalance("0xbFDf092Ea03FC2ff3f9D0a8aFf0F567638209C0F")